In [38]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

url = "https://www.foodsafetykorea.go.kr/portal/healthyfoodlife/searchHomeHF.do?menu_grp=MENU_NEW01&menu_no=2823"
url2 = "https://www.foodsafetykorea.go.kr/portal/healthyfoodlife/searchHomeHFDetail.do"

detailURL = 'https://www.foodsafetykorea.go.kr/portal/healthyfoodlife/searchHomeHFDetail.do'

html = urlopen(url)
bsObject = BeautifulSoup(html, "html.parser")

print(bsObject)



<!DOCTYPE html>

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible" id="document_mode"/>
<meta content="width=device-width,initial-scale=1.0,minimum-scale=1.0,maximum-scale=1.0" name="viewport">
<title id="brwsTitle">식품안전나라</title>
<link href="/css/2021/sub_2021.css" rel="stylesheet" type="text/css"/>
<link href="/css/2021/search.css" rel="stylesheet" type="text/css"/>
<link href="/js/2021/jquery.scrollbar.css" rel="stylesheet" type="text/css"/>
<link href="/css/jquery.selecter.css" rel="stylesheet" type="text/css"/>
<link href="/css/jquery.mCustomScrollbar.css" rel="stylesheet" type="text/css"/>
<script src="/js/2021/jquery-1.11.3.min.js" type="text/javascript"></script>
<script src="/js/2021/jquery-ui.min.js" type="text/javascript"></script>
<script src="/js/2021/jquery.easing.1.3.js" type="text/javascript"></script>
<script src="/js/2021/common.js" type="text/javascript"></script

In [106]:
import requests
import json
import pandas as pd


url = 'https://www.foodsafetykorea.go.kr/portal/healthyfoodlife/searchHomeHFProc.do'

#50개 가져옴
i=50

dic={"menu_no":"2823",
     "menu_grp": "MENU_NEW01",
     "menuNm":"",
     "copyUrl":"https://www.foodsafetykorea.go.kr:443/portal/healthyfoodlife/searchHomeHF.do?menu_grp=MENU_NEW01&menu_no=2823",
     "mberld":"",
     "mberNo":"",
     "favorListCnt":"0",
     "search_code":"01",
     "search_word":"",
     "start_idx": "1",
     "show_cnt":str(i)
    }

result = []
res = requests.post(url, data=dic)
result += json.loads(res.text)


df2 = pd.DataFrame(result)
prd_no_list = df2['prdlst_report_ledg_no']
#print(df)    
print(prd_no_list)
#df.to_excel("test1.xlsx")

0     2021021000400614
1     2021021000400388
2     2021021000400617
3     2021021000400618
4     2021021000400615
5     2021021000400938
6     2021021000400619
7     2021021000400389
8     2021021000400084
9     2021021000398811
10    2021021000398580
11    2021021000398581
12    2021021000398582
13    2021021000398583
14    2021021000397166
15    2021021000398572
16    2021021000398801
17    2021021000398802
18    2021021000398800
19    2021021000398803
20    2021021000396898
21    2021021000396901
22    2021021000395646
23    2021021000395944
24    2021021000395642
25    2021021000395644
26    2021021000395945
27    2021021000395643
28    2021021000395940
29    2021021000395943
30    2021021000395946
31    2021021000395942
32    2021021000395941
33    2021021000395948
34    2021021000396178
35    2021021000395789
36    2021021000395630
37    2021021000395629
38    2021021000395632
39    2021021000395634
40    2021021000395624
41    2021021000395633
42    2021021000395627
43    20210

In [108]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup

detailURL = 'https://www.foodsafetykorea.go.kr/portal/healthyfoodlife/searchHomeHFDetail.do'

dic={
    "prdlstReportLedgNo": prd_no_list[0],
     "search_word":"",
     "search_code":"01",
     "start_idx": "",
     "menu_no":"2823",
     "menu_grp": "MENU_NEW01",
    }

result = []
res = requests.post(detailURL, data=dic)
html_doc = res.text
soup = BeautifulSoup(html_doc, 'html.parser')
#print(soup.prettify())

cindex = []

for title in soup.find_all('th'):
    cindex.append(title.get_text()) 

cindex = cindex[2:-6]    
#print(cindex)

df = pd.DataFrame(columns=cindex)
#print(df)
for prdNo in prd_no_list:
    dic={
    "prdlstReportLedgNo": prdNo,
     "search_word":"",
     "search_code":"01",
     "start_idx": "",
     "menu_no":"2823",
     "menu_grp": "MENU_NEW01",
    }
    result = []
    res = requests.post(detailURL, data=dic)
    html_doc = res.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    datalist = []
    for DATA in soup.find_all('td'):
        #print(DATA.get_text())
        datalist.append(DATA.get_text())
    datalist = datalist[:-3]
    #print(datalist)
    temp = pd.DataFrame(data= [datalist], columns = cindex)
    df = df.append(temp)
#print(df)
df = df.drop('등록일자', axis=1)
df = df.drop('유통기한', axis=1)
df = df.drop('포장재질(방법)', axis=1)
df = df.drop('보존 및 유통기준', axis=1)
df = df.drop('기능성 내용', axis=1)
df = df.drop('기준 및 규격', axis=1)

df['URL'] = url
df['추출 후 남은 데이터'] = "#none" #요건 그냥 추가 ㅎ 
df.rename(columns={'업소명':'제조사','신고번호':'제품코드','성상':'제형',
                   '섭취량/섭취 방법':'섭취방법','섭취시주의사항':'주의 사항'}, inplace =True)
#df = df[['제품코드','제품명','제조사','제형','섭취방법','주의 사항','URL','추출 후 남은 데이터']]

#원재료명 가져오기
rawURL = 'https://www.foodsafetykorea.go.kr/portal/healthyfoodlife/searchHfPrdlstRawmtrl.do'

rawList=[]

for CODE in df['제품코드']:
    dic={"prdlst_report_no": str(CODE)
        }
    result = []
    res = requests.post(rawURL, data=dic)
    result += json.loads(res.text)
    #print(result)
    df3 = pd.DataFrame(result)
    rString=[]
    for raws in df3['rawmtrl_nm']:
        rString.append(raws)
    rString = ",".join(rString)
    rawList.append(rString)

df['원재료명'] = rawList

cutlist=[]
for type in df['제형']:
    cutlist.append( type.split()[-1] )
#print(cutlist)
df['제형'] = cutlist

foodcode=[]
for type in df['제품코드']:
    foodcode.append("kfood-"+str(type))
df['제품코드'] = foodcode


df = df[['제품코드','제품명','제조사','제형','섭취방법','주의 사항','원재료명','URL','추출 후 남은 데이터']]

df.to_excel("result.xlsx", index = False )

print("완료")

완료
